In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [3]:
embeddings_model = OpenAIEmbeddings()     # 임베딩 모델을 정의합니다.

In [4]:
# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [8]:
# 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위해서입니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [11]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import VectorStoreRetrieverMemory

In [12]:
llm = ChatOpenAI(api_key=key, model_name="gpt-4o-mini", temperature=0)

In [13]:
# ConversationChain을 생성합니다. 
conversation = ConversationChain(
    llm=llm,                            
    memory=VectorStoreRetrieverMemory(retriever=retriever)
)

C:\Users\lee\AppData\Local\Temp\ipykernel_3932\1877807336.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [14]:
conversation.predict(    
    input="안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
)

'안녕하세요! 면접에 참석하게 되어 정말 기쁩니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 이름은 ChatGPT이며, 자연어 처리 기술을 기반으로 하고 있습니다. 여러 분야에 대한 지식을 가지고 있으며, 질문에 대한 답변을 제공하거나, 아이디어를 제안하거나, 심지어 창작 활동도 도와드릴 수 있습니다. 여러분과의 대화를 통해 더 많은 것을 배우고, 도움이 될 수 있기를 기대합니다! 여러분에 대해 더 알고 싶습니다. 어떤 질문이든지 편하게 해주세요!'

In [15]:
conversation.predict(    
    input="프로젝트에서 어떤 역할을 맡았나요?"
)

'저는 프로젝트에서 주로 정보 제공과 아이디어 제안 역할을 맡고 있습니다. 예를 들어, 팀원들이 특정 주제에 대해 논의할 때, 관련된 데이터나 연구 결과를 찾아 제공하거나, 문제 해결을 위한 다양한 접근 방식을 제안할 수 있습니다. 또한, 창의적인 글쓰기나 콘텐츠 생성이 필요할 때도 도움을 줄 수 있습니다. 여러분의 프로젝트에 어떤 도움이 필요하신가요? 더 구체적으로 말씀해 주시면, 더욱 유용한 정보를 제공해 드릴 수 있을 것 같습니다!'

In [19]:
conversation.predict(    
    input="팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
)

'팀 프로젝트에서 어려움을 겪었던 경험이 있다면, 저는 주로 소통과 협업을 통해 문제를 해결하려고 합니다. 예를 들어, 팀원 간의 의견 차이나 갈등이 발생했을 때, 저는 중재자의 역할을 맡아 각자의 의견을 경청하고, 공통의 목표를 다시 상기시키며 대화를 이끌어 나갑니다.\n\n또한, 문제가 발생한 원인을 분석하고, 해결책을 모색하기 위해 팀원들과 브레인스토밍 세션을 진행하기도 합니다. 이 과정에서 각자의 강점을 활용하여 문제를 해결할 수 있는 방법을 찾는 것이 중요하다고 생각합니다.\n\n혹시 특정한 상황에 대한 조언이 필요하신가요? 더 구체적인 사례를 말씀해 주시면, 그에 맞는 해결책을 제안해 드릴 수 있을 것 같습니다!'

In [21]:
# 다음의 질문을 했을 때 Vector Store 로 부터 1개(k=1 이기 때문)의 가장 관련성 높은 대화를 반환합니다.
# 질문: "면접자 전공은 무엇인가요?"

# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(conversation.memory.load_memory_variables({"prompt": "면접자 전공은 무엇인가요?"})["history"])

input: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
response: 안녕하세요! 면접에 참석하게 되어 정말 기쁩니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 이름은 ChatGPT이며, 자연어 처리 기술을 기반으로 하고 있습니다. 여러 분야에 대한 지식을 가지고 있으며, 질문에 대한 답변을 제공하거나, 아이디어를 제안하거나, 심지어 창작 활동도 도와드릴 수 있습니다. 여러분과의 대화를 통해 더 많은 것을 배우고, 도움이 될 수 있기를 기대합니다! 여러분에 대해 더 알고 싶습니다. 어떤 질문이든지 편하게 해주세요!


In [22]:
# 이번에는 다른 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
# 질문: "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"

print(conversation.memory.load_memory_variables({"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"})["history"])

input: 프로젝트에서 어떤 역할을 맡았나요?
response: 저는 프로젝트에서 주로 정보 제공과 아이디어 제안 역할을 맡고 있습니다. 예를 들어, 팀원들이 특정 주제에 대해 논의할 때, 관련된 데이터나 연구 결과를 찾아 제공하거나, 문제 해결을 위한 다양한 접근 방식을 제안할 수 있습니다. 또한, 창의적인 글쓰기나 콘텐츠 생성이 필요할 때도 도움을 줄 수 있습니다. 여러분의 프로젝트에 어떤 도움이 필요하신가요? 더 구체적으로 말씀해 주시면, 더욱 유용한 정보를 제공해 드릴 수 있을 것 같습니다!
